#Conectamos a google drive e importamos librerias

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
!pip install pandas_summary
from pandas_summary import DataFrameSummary
import numpy as np
import datetime
!pip install isoweek
from isoweek import Week
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
!pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from random import sample

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

# Para importar el Helper
%cd "/gdrive/Shareddrives/Redes neuronales/TP2"
!pip install import-ipynb
import import_ipynb
from Helper import *

/gdrive/Shareddrives/Redes neuronales/TP2
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=b2a3f6b24cea1c01e030b70ccc4a6fdbf212ddbf90f7887108b47ab55383a6b5
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from Helper.ipynb


# Se cargan los archivos preprocesados

In [6]:
df = pd.read_feather('data preprecesada/train_normalized_data_!sales0.fth')
df_test = pd.read_feather('data preprecesada/test_normalized_data.fth')

In [7]:
len(df.columns)

89

In [8]:
df.columns

Index(['index', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week',
       'Day', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitatio

# Listamos las variables

##Variables categóricas

In [9]:
 cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day',
             'StateHoliday', 'SchoolHoliday', 'Promo2SinceWeek',
             'CompetitionMonthsOpen', 'Promo2Weeks', 'Promo2',
             'StoreType', 'Assortment', 'PromoInterval', 
             'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 'Week',
              'Promo', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw',
             'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw',
             'Close_bool_bw','Close_bool_fw']

In [10]:
np.unique(df['Promo'])

array([0, 1])

## Variables continuas

Se eligen las caracteristicas del clima que se consideran relevantes

In [11]:
contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 
               'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool',
    'AfterPromo','BeforePromo',
   'StateHoliday_bool','AfterSchoolHoliday','BeforeSchoolHoliday',
   'AfterClose_bool','BeforeClose_bool']

In [12]:
len(contin_vars)+len(cat_vars)

50

In [13]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]
uniques

,Store,DayOfWeek,Year,Month,Day,StateHoliday,SchoolHoliday,Promo2SinceWeek,CompetitionMonthsOpen,Promo2Weeks,Promo2,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Promo,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw,Close_bool_bw,Close_bool_fw
uniques,1115,7,3,12,31,4,2,24,25,26,2,4,3,4,21,8,12,52,2,22,6,6,3,3,8,8,7,7


# Corremos los modelos

##Asignación de embeddings

In [14]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 3,'Promo':2, 'Year': 3, 'Month': 4,
'Day': 10, 'StateHoliday': 3,'SchoolHoliday':2, 'Promo2SinceWeek': 3,
'CompetitionMonthsOpen': 4, 'Promo2Weeks': 3,  'Promo2':2, 'StoreType': 2,
 'Assortment': 3, 'PromoInterval': 3,'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 
 'State': 4, 'Events': 7, 'Promo_fw': 4,'Promo_bw': 4, 'Week':10,
 'StateHoliday_bool_fw': 2, 'StateHoliday_bool_bw': 2, 'SchoolHoliday_fw': 2,
'SchoolHoliday_bw': 2,'Close_bool_bw':3,'Close_bool_fw':3}

##Modelo

In [15]:
# Parametros de entrada para creación de las capas ocultas:
cant_capas_ocultas = 2
cantidad_neuronas = [1000, 500]
l2_lambda = 1e-3
activation_function = ["relu","relu"]
alpha = [0, 0] # Se puede cambiar el valor de alpha en caso de usar función LeakyRelu.
# Recordar que el alpha tiene que ser un vector
kernel_initializer = "uniform"

# Parametros de entrada para creación modelo entero:
add_customers = True
output_activation = 'linear'

all_vars = cat_vars + contin_vars #variables elegidas

log_output = False

lr=0.001

metrics=['mse', rmspe]
loss='mse'
epochs = 10
batch_size = 256
verbose=2

df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

Hacemos que le entren 10 características categóricas y 10 características continuas por lo menos. 

In [16]:
cat_vars_random = [sample(cat_vars,np.random.randint(10,len(cat_vars))) for i in range(10)]

In [17]:
contin_vars_random = [sample(contin_vars,np.random.randint(10,len(contin_vars))) for i in range(10)]

In [ ]:
res_metrics = pd.DataFrame(columns=['Cantidad de características','Caracteristicas modelo','Puntaje (Sales_rmspe)'])
for i in cat_vars_random:

  cat_vars_chosen = i

  print(f'Cat vars chosen: {cat_vars_chosen}')
  cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model2(cat_vars_chosen, uniques, cat_var_dict)

  for j in contin_vars_random:

    contin_vars_chosen = j 

    print(f'Cont vars chosen: {contin_vars_chosen}')
    cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars_chosen)

    print("Creamos los embeddings y layers")
    concatenate_layer = MLP_layers(cat_vars_embed_outs, cont_vars_inputs, l2_lambda, kernel_initializer, cant_capas_ocultas, cant_neuronas= cantidad_neuronas, activation_func= activation_function)
    print(concatenate_layer)

    print(f'Creamos el modelo {len(res_metrics)}')
    print(cat_var_inputs)
    model = crear_MLP(add_customers, cat_var_inputs, cont_vars_inputs, concatenate_layer, output_activation)

    all_vars = cat_vars_chosen + contin_vars_chosen

    print("Normalizamos el dataset")
    X_train,X_val,X_test,y_train,y_val,y_norm = obtener_x_y(df_train,df_val,df_test,all_vars,add_customers=add_customers,log_output=log_output)  

    model_chosen=f'modelos generados/eleccion_params/bestmodel{len(res_metrics)}.hdf5'

    print(f'Entrenamos el modelo {len(res_metrics)}')
    
    history,model = entrenar_MLP(X_train,X_val,y_train,y_val,all_vars,model=model,add_customers=add_customers,log_output=log_output,lr=lr,metrics=metrics,loss=loss,model_chosen=model_chosen,epochs = epochs,batch_size = batch_size,verbose=verbose)  

    print("Obtenemos las métricas")
    metric_cross = metricas_MLP(X_val,y_val,model,model_chosen=model_chosen)    
    puntaje = metric_cross['Sales_rmspe']

    data_columns_df = dict(zip(res_metrics.columns,[len(all_vars),all_vars,puntaje]))
    res_metrics = res_metrics.append(data_columns_df,ignore_index=True)

res_metrics.to_csv("/gdrive/Shareddrives/Redes neuronales/TP2/resultados/results_eleccion.csv")

Cat vars chosen: ['Store', 'CompetitionOpenSinceYear', 'StateHoliday', 'Promo_fw', 'Events', 'Promo2SinceYear', 'Promo2SinceWeek', 'StoreType', 'SchoolHoliday_fw', 'State', 'Day', 'Close_bool_fw', 'StateHoliday_bool_fw', 'Close_bool_bw']
Cont vars chosen: ['BeforeSchoolHoliday', 'Max_TemperatureC', 'trend_DE', 'AfterPromo', 'Mean_Wind_SpeedKm_h', 'CompetitionDistance', 'AfterSchoolHoliday', 'BeforeStateHoliday_bool', 'Min_TemperatureC', 'Max_Wind_SpeedKm_h', 'BeforeClose_bool', 'Mean_TemperatureC', 'CloudCover', 'StateHoliday_bool', 'BeforePromo', 'AfterStateHoliday_bool', 'trend', 'Max_Humidity']
Creamos los embeddings y layers
KerasTensor(type_spec=TensorSpec(shape=(None, 500), dtype=tf.float32, name=None), name='activation_1/Relu:0', description="created by layer 'activation_1'")
Creamos el modelo 0
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Store_input')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'CompetitionOpenSinceYear_input')>, <KerasTe

#Se levantan todos los resultados csv


Se unen en un solo dataframe todos los archivos de csv que se generaron en los otros scripts de "Eleccion caracteristicas y parametros"

In [41]:
results_metrics = pd.read_csv('/gdrive/Shareddrives/Redes neuronales/TP2/resultados/results_eleccion.csv', low_memory=False)
results_metrics["Num_Corrida"] = 0

In [42]:
results_metrics

,Unnamed: 0,Cantidad de características,Caracteristicas modelo,Puntaje (Sales_rmspe),Num_Corrida
0,0,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.164624,0
1,1,28,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.225814,0
2,2,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160013,0
3,3,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160265,0
4,4,35,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.149459,0
5,5,35,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.148995,0
6,6,26,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.162675,0
7,7,24,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.147781,0
8,8,24,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.154298,0
9,9,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.150360,0


Se juntan todos los resultados en un solo DataFrame para evaluar cual es el que mejor resultados dio.

In [43]:
path = '/gdrive/Shareddrives/Redes neuronales/TP2/eleccion caracteristicas/resultados/results_eleccion'
for i in range(1,13):
  path_name = f'{path}{i}.csv'
  new_csv = pd.read_csv(path_name, low_memory=False)
  new_csv["Num_Corrida"] = i
  results_metrics = pd.concat([results_metrics, new_csv], axis=0)

In [44]:
results_metrics

,Unnamed: 0,Cantidad de características,Caracteristicas modelo,Puntaje (Sales_rmspe),Num_Corrida
0,0,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.164624,0
1,1,28,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.225814,0
2,2,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160013,0
3,3,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160265,0
4,4,35,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.149459,0
...,...,...,...,...,...
4,4,32,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.134456,12
5,5,34,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.136274,12
6,6,21,"['StateHoliday_bool_bw', 'Promo2SinceYear', 'P...",0.147780,12
7,7,23,"['StateHoliday_bool_bw', 'Promo2SinceYear', 'P...",0.142965,12


Se re definen los indices del DataFrame

In [45]:
results_metrics.reset_index(inplace=True, drop=True)
results_metrics

,Unnamed: 0,Cantidad de características,Caracteristicas modelo,Puntaje (Sales_rmspe),Num_Corrida
0,0,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.164624,0
1,1,28,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.225814,0
2,2,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160013,0
3,3,32,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.160265,0
4,4,35,"['Store', 'CompetitionOpenSinceYear', 'StateHo...",0.149459,0
...,...,...,...,...,...
133,4,32,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.134456,12
134,5,34,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.136274,12
135,6,21,"['StateHoliday_bool_bw', 'Promo2SinceYear', 'P...",0.147780,12
136,7,23,"['StateHoliday_bool_bw', 'Promo2SinceYear', 'P...",0.142965,12


In [46]:
results_metrics.sort_values('Puntaje (Sales_rmspe)', ascending= True, inplace= True)
results_metrics

,Unnamed: 0,Cantidad de características,Caracteristicas modelo,Puntaje (Sales_rmspe),Num_Corrida
65,8,42,"['Close_bool_bw', 'Assortment', 'CompetitionOp...",0.132160,4
94,1,40,"['CompetitionOpenSinceYear', 'Day', 'StateHoli...",0.133824,8
133,4,32,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.134456,12
128,8,43,"['StateHoliday_bool_bw', 'Store', 'Close_bool_...",0.135078,11
55,7,42,"['Assortment', 'State', 'Promo_fw', 'Competiti...",0.135971,3
...,...,...,...,...,...
111,0,38,"['Promo2SinceYear', 'StateHoliday_bool_fw', 'S...",0.448197,10
38,8,25,"['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...",0.458494,1
108,6,25,"['CompetitionOpenSinceYear', 'Promo_fw', 'Day'...",0.460379,9
37,7,27,"['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...",0.470376,1


Se puede ver que los resultados dependen mucho de las caracteristicas que se le ponga especificamente y no de la cantidad en si. Sin embargo, todos los primeros mejores resultados se ve que evaluan con más de 30 caracteristicas.

In [47]:
results_metrics.to_csv("/gdrive/Shareddrives/Redes neuronales/TP2/resultados/results_combinaciones_completas.csv")

In [51]:
results_metrics

,Unnamed: 0,Cantidad de características,Caracteristicas modelo,Puntaje (Sales_rmspe),Num_Corrida
65,8,42,"['Close_bool_bw', 'Assortment', 'CompetitionOp...",0.132160,4
94,1,40,"['CompetitionOpenSinceYear', 'Day', 'StateHoli...",0.133824,8
133,4,32,"['CompetitionMonthsOpen', 'CompetitionOpenSinc...",0.134456,12
128,8,43,"['StateHoliday_bool_bw', 'Store', 'Close_bool_...",0.135078,11
55,7,42,"['Assortment', 'State', 'Promo_fw', 'Competiti...",0.135971,3
...,...,...,...,...,...
111,0,38,"['Promo2SinceYear', 'StateHoliday_bool_fw', 'S...",0.448197,10
38,8,25,"['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...",0.458494,1
108,6,25,"['CompetitionOpenSinceYear', 'Promo_fw', 'Day'...",0.460379,9
37,7,27,"['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...",0.470376,1


In [52]:
results_metrics['Caracteristicas modelo']

65     ['Close_bool_bw', 'Assortment', 'CompetitionOp...
94     ['CompetitionOpenSinceYear', 'Day', 'StateHoli...
133    ['CompetitionMonthsOpen', 'CompetitionOpenSinc...
128    ['StateHoliday_bool_bw', 'Store', 'Close_bool_...
55     ['Assortment', 'State', 'Promo_fw', 'Competiti...
                             ...                        
111    ['Promo2SinceYear', 'StateHoliday_bool_fw', 'S...
38     ['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...
108    ['CompetitionOpenSinceYear', 'Promo_fw', 'Day'...
37     ['Events', 'SchoolHoliday_fw', 'Promo_fw', 'We...
110    ['CompetitionOpenSinceYear', 'Promo_fw', 'Day'...
Name: Caracteristicas modelo, Length: 138, dtype: object

In [56]:
results_metrics['Cantidad de características'][94]

28

In [58]:
results_metrics['Caracteristicas modelo'][94]

"['CompetitionOpenSinceYear', 'Day', 'StateHoliday_bool_bw', 'Promo2', 'StateHoliday_bool_fw', 'PromoInterval', 'Month', 'Promo2SinceYear', 'Close_bool_bw', 'Week', 'Promo2SinceWeek', 'Events', 'StateHoliday', 'State', 'Year', 'CompetitionMonthsOpen', 'Promo2Weeks', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_fw', 'DayOfWeek', 'Store', 'SchoolHoliday_bw', 'Min_TemperatureC', 'CloudCover', 'AfterPromo', 'Mean_TemperatureC', 'Max_Wind_SpeedKm_h', 'BeforeSchoolHoliday', 'StateHoliday_bool', 'AfterSchoolHoliday', 'Min_Humidity', 'AfterClose_bool', 'trend_DE', 'BeforeStateHoliday_bool', 'Max_Humidity', 'CompetitionDistance', 'Max_TemperatureC', 'BeforePromo', 'trend']"

# Evaluacion de caracteristicas

## Se elige el archivo con el mejor Sales_rmspe

In [26]:
print(f'Minimo valor de Sales_rmspe: {results_metrics["Puntaje (Sales_rmspe)"].min()}')
print(f'Index del minimo valor de Sales_rmspe: {results_metrics["Puntaje (Sales_rmspe)"].idxmin()}')

Minimo valor de Sales_rmspe: 0.1321597844362259
Index del minimo valor de Sales_rmspe: 65


Detalle de las caracteristicas que mejores resultados dieron:

In [27]:
index = results_metrics["Puntaje (Sales_rmspe)"].idxmin()
best_caracteristicas = results_metrics['Caracteristicas modelo'][index]
num_corrida = results_metrics['Num_Corrida'][index]
num_modelo = results_metrics['Unnamed: 0'][index]

In [28]:
print(best_caracteristicas)
print(f'El modelo a utilizar es el de: results_eleccion{num_corrida}, modelo {num_modelo}')

['Close_bool_bw', 'Assortment', 'CompetitionOpenSinceYear', 'DayOfWeek', 'StateHoliday_bool_fw', 'State', 'SchoolHoliday', 'Year', 'Week', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'StoreType', 'Promo2Weeks', 'Events', 'Day', 'Month', 'Close_bool_fw', 'CompetitionMonthsOpen', 'StateHoliday', 'Promo', 'Promo2', 'PromoInterval', 'Promo2SinceWeek', 'StateHoliday_bool_bw', 'Store', 'Promo2SinceYear', 'AfterSchoolHoliday', 'Mean_TemperatureC', 'AfterStateHoliday_bool', 'AfterPromo', 'Max_TemperatureC', 'Mean_Humidity', 'BeforeStateHoliday_bool', 'Max_Wind_SpeedKm_h', 'trend', 'CompetitionDistance', 'AfterClose_bool', 'trend_DE', 'Precipitationmm', 'CloudCover', 'Min_TemperatureC']
El modelo a utilizar es el de: results_eleccion4, modelo 8


Se toma la mejor combinación de parametros y caracteristicas para hacer la evaluación del modelo con estás.

Este submit va a estar en **TP2-Modelo**